In [ ]:
import pandas as pd
import numpy as np
import itertools
import re

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib import cm
from datetime import datetime
from ipywidgets import widgets  
from IPython.display import display


from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.metrics import log_loss

from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

import xgboost as xgb

print "imports imported"

In [ ]:
crimeData = pd.read_csv('train.csv')
Categories = ['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
              'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
              'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
              'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
              'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
              'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
              'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
              'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
              'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
              'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
              'WARRANTS', 'WEAPON LAWS']

In [ ]:
def recodeData(df, isTrain = False):
    '''This function takes in the dataframe that we get from loading in the 
    SF crime data and returns a re-coded dataframe that has all the 
    additional features we want to add and the categorical features recoded 
    and cleaned.
    '''

    #since the modifications are done in-place we don't return the dataframe. 
    #we do, however, return the list of all the columns we added.
    df, newLatLon = removeOutlierLatLon(df)
    df, newDate = recodeDates(df)
    df, newDistrict = recodePoliceDistricts(df)
    df, newWeather = addWeather(df)
    print "recoding addresses"
    df, newAddress, streetColumns = recodeAddresses(df)

    
    addedColumns = [] 
    addedColumns += newDate
    addedColumns += newDistrict 
    addedColumns += newLatLon
    addedColumns += newWeather
    addedColumns += newAddress
    
    
    
   

    if (isTrain):
        newCategory = recodeCategories(df)
        addedColumns += newCategory
        try: #prevents error if the coumns have already been removed or we are processing test data
            columnsToDrop = ['Descript', 'Resolution']
            df.drop(columnsToDrop, axis=1, inplace=True)
        except:
            print "already recoded or using test data"
         

    return df, addedColumns, streetColumns

In [ ]:
def recodeDates(df):
    '''This function takes in a dataframe and recodes the date field into 
    useable values. Here, we also recode the day of week.'''
    #Recode the dates column to year, month, day and hour columns
    df['DateTime'] = df['Dates'].apply(
        lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

    df['Year'] = df['DateTime'].apply(lambda x: x.year)
    df['Month'] = df['DateTime'].apply(lambda x: x.month)
    df['Day'] = df['DateTime'].apply(lambda x: x.day)
    df['Hour'] = df['DateTime'].apply(lambda x: x.hour)
    df['Minute'] = df['DateTime'].apply(lambda x: x.minute)
    df['DayOfWeekRecode'] = df['DateTime'].apply(lambda x: x.weekday())
    df['MinuteOfWeek'] = df['DateTime'].apply(lambda x: x.weekday()*24*60 + x.hour*60 + x.minute)

    return df, ['Year', 'Month', 'Day', 'Hour', 'Minute', 'DayOfWeekRecode','MinuteOfWeek']

In [ ]:
def recodePoliceDistricts(df):
    '''This function recodes the police district to a one-hot encoding 
    scheme.'''
    districts = df['PdDistrict'].unique().tolist()
    newColumns = []
    for district in districts:
        newColumns.append('District' + district)
        df['District' + district] = df['PdDistrict'].apply(
            lambda x: int(x == district))

    return df, newColumns

In [ ]:
def recodeAddresses(df):
    '''This function will attempt to create some features related to the address field in the database. To do this, 
    first, we need to split up the address field into two different address fields'''
    
    #If there are two addresss, split fields. Also extract the block number
    df['Address1'] = df['Address'].apply(lambda x: re.sub(r'^\d+ Block of ','',x.split(" / ")[0]))
    df['Address2'] = df['Address'].apply(lambda x: (x.split(" / ")[1]) if (len(x.split(" / ")) > 1) else '')
    
    streets = set(df['Address1'].unique().tolist() + df['Address2'].unique().tolist())
    
    streetColumns = []
    i = 0
    
    print "starting address dummy creation"
    
    #address1Dummy = pd.get_dummies(df['Address1']).rename(columns=lambda x: str(x))
    address1Dummy = pd.get_dummies(df['Address1'])
    address1Dummy = address1Dummy.replace(0, np.nan)
    
    print "completed address 1 dummy creation"
    address2Dummy = pd.get_dummies(df['Address2'])
    address2Dummy = address2Dummy.replace(0, np.nan)
    
    print "completed address 2 dummy creation"
    
#     print address1Dummy
#     print address2Dummy.info()
    
    mergedAddressDummy = address1Dummy.combine_first(address2Dummy)
    
    print "completed address dummy DataFrames merge"
    mergedAddressDummy = mergedAddressDummy.fillna(0)
    print "completed fillna on mergedAddressDummy"
    
    streetColumns = list(mergedAddressDummy.columns.values)
    
    df = pd.concat([df, mergedAddressDummy], axis=1)
    print "completed merge of original df and new dummy variable df"
    
    print "Dummy creation finished"
    
#     for street in streets:
#         df['OnStreet' + street] = df.apply(lambda x: (x['Address1'] == street or x['Address2'] == street), axis=1)
#         streetColumns.append('OnStreet' + street)
#         i += 1
#         print i
        

    
    df['BlockNumber'] = df['Address'].apply(lambda x: int(re.findall(r'^\d+',x)[0]) if (len(re.findall(r'^\d+',x)) > 0) else None )
    df['BlockNumber'] = df['BlockNumber'].fillna(-1)
    
    #Also add the "did the crime occur on a street corner field?"
    df['StreetCornerFlag'] = df['Address'].apply(lambda x: len(x.split(" / ")) > 1)
    
    return df, ['StreetCornerFlag', 'BlockNumber'], streetColumns

In [ ]:
def removeOutlierLatLon(df):
    '''This function will attempt remove outlier Latitudes and Longitudes'''
    df.loc[df.X > -121, 'X'] = df.loc[(df.X > -121)].apply(lambda row: df.X[df["PdDistrict"] == row['PdDistrict']].median(), axis=1)
    df.loc[df.Y > 38, 'Y'] = df.loc[(df.Y > 38)].apply(lambda row: df.Y[df["PdDistrict"] == row['PdDistrict']].median(), axis=1)

    return df, ['X', 'Y']

In [ ]:
def addWeather(df):
    df['DATE'] = df['DateTime'].apply(lambda x: int( str(x.year)+x.strftime('%m')+x.strftime('%d') ))
    
    weatherData = pd.read_csv('weather1.csv')
    weatherData = weatherData.replace('-9999', np.nan)
    
    weatherData = weatherData[['DATE','PRCP','TMAX','TMIN']]
    
    df = pd.merge(df, weatherData, on='DATE')
    
    return df, ['PRCP','TMAX','TMIN']

In [ ]:
def recodeCategories(df):
    '''This function will attempt remove outlier Latitudes and Longitudes'''
    #if 'Category' in df.columns:
    df['CategoryRecode'] = df.Category.apply(lambda x: Categories.index(x))
        
    return df, ['CategoryRecode']

In [ ]:
crimeData, addedColumns, streetColumns = recodeData(
    crimeData, isTrain = True)
crimeData.describe()

In [ ]:
columnsToUse = addedColumns

commonStreets = ['FOLSOM ST','16TH ST','JONES ST','TAYLOR ST',
                 'ARMSTRONG AV','EDDY ST','LARKIN ST','CASTRO ST',
                 '10TH AV','5TH ST','HAIGHT ST','OFARRELL ST',
                 '11TH AV','PAGE ST','FITCH ST','CAPP ST','13TH ST',
                 '24TH AV','17TH ST','18TH ST','19TH ST','GENEVA AV',
                 'GEARY BL','BRYANT ST','HYDE ST','4TH ST','FULTON ST',
                 'LEAVENWORTH ST','COLE ST','ALEMANY BL','PHELPS ST',
                 'MISSION ST','6TH ST','12TH AV','SHOTWELL ST',
                 'TREAT AV','7TH ST','JEFFERSON ST','QUESADA AV',
                 'TURK ST','2ND ST','MARKET ST','GGBRIDGE HY',
                 '24TH ST','CAPITOL AV','KEARNY ST','HARRISON ST',
                 'LYON ST','BUSH ST','POLK ST','3RD ST','ELLIS ST',
                 'SOUTH VAN NESS AV','POTRERO AV','20TH ST','POWELL ST']

columnsToUse = ['X','Y', 'Year', 'Month', 'Day','Hour', 'Minute',
       'DayOfWeekRecode', 'DistrictNORTHERN', 'DistrictPARK',
       'DistrictINGLESIDE', 'DistrictBAYVIEW', 'DistrictRICHMOND',
       'DistrictCENTRAL', 'DistrictTARAVAL', 'DistrictTENDERLOIN',
       'DistrictMISSION', 'DistrictSOUTHERN', 'StreetCornerFlag', 'PRCP','TMAX','TMIN']

# columnsToUse = ['X','Y', 'Year', 'Month', 'Day','Hour', 'Minute',
#        'DayOfWeekRecode', 'StreetCornerFlag']

X = crimeData[columnsToUse]
y = crimeData['CategoryRecode']

print "prepping for training complete"

In [ ]:
def runOnTrainData(clf, numSplits=1):
    
    k_folds = StratifiedShuffleSplit(y, numSplits, test_size=0.5, random_state=0)

    scores = []
    print "starting kfold testing"

    for k, (train, test) in enumerate(k_folds):
        print "starting fit"
        start = time()
        clf.fit(X.iloc[train], y.iloc[train], eval_metric = 'mlogloss', verbose=3)
        print "fit complete, time: " + str((time() - start))
        startPredictTime = time()
        probs = clf.predict_proba(X.iloc[test])
        print "predict complete, time: " + str((time() - startPredictTime))
        score = log_loss(y.iloc[test].values, probs)
        print score
        print "total time: " + str((time() - start))
        scores.append(score)

    print(scores)
    print("Average: " + str(np.average(scores)))

In [ ]:
clf = xgb.XGBClassifier(max_depth=8, n_estimators=800, objective='multi:softprob', max_delta_step = .8, learning_rate = .0375, nthread = -1)
runOnTrainData(clf, numSplits=2)

In [ ]:
predictorsUsed = ['DistrictBAYVIEW',
 'Y',
 'Hour',
 'DistrictSOUTHERN',
 'DistrictTARAVAL',
 'DistrictRICHMOND',
 'Month',
 'DistrictTENDERLOIN',
 'DistrictNORTHERN',
 'DayOfWeekRecode',
 'Minute',
 'Year',
 'StreetCornerFlag',
 'X',
 'DistrictPARK',
 'DistrictMISSION',
 'Day',
 'DistrictCENTRAL',
 'DistrictINGLESIDE']

In [ ]:
def make_submission(clf, Xtrain, ytrain, predictors, path='my_submissionXGBoostWithWeather.csv'):
    '''This function will take in a trained model, a list of predictors, and an optional 
    filepath and create a submissision file for us.'''
   
    test_data = pd.read_csv('test.csv')
    print "test data loaded"
    
    test_data, newColumns, streetColumns = recodeData(test_data)
    print "test data recoded"
    
    testDataColumns = list(test_data.columns.values)
    
    existingPredictors = list(set(predictors) & set(testDataColumns))
    
    clf.fit(Xtrain[existingPredictors], ytrain, eval_metric = 'mlogloss')
    print "model fitted with all data"
    
    #clf.fit(trainX[predictors], trainY)
    predictions = clf.predict_proba(test_data[existingPredictors])

    submission = pd.DataFrame({
        'Id': test_data.Id
    })
    
    for i in range(predictions.shape[1]):
        submission[Categories[i]] = predictions[:,i]
    submission.to_csv(path, index=False)

    print(" -- Wrote submission to file {}.".format(path))
    return existingPredictors

In [ ]:
predictorsUsed = make_submission(clf, X, y, columnsToUse)

In [ ]:
# Utility function to report best scores
def report(grid_scores, n_top=10):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

clfOpt = xgb.XGBClassifier(objective='multi:softprob', nthread = 12, max_delta_step = 1)
#(max_depth=7, n_estimators=100, objective='multi:softprob', max_delta_step = 1, learning_rate = .33)

param_dist = {"max_depth": [5, 7, 9],
              "n_estimators": [500, 1000],
              "learning_rate": [0.033, .1]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clfOpt, param_distributions=param_dist,
                                   n_iter=n_iter_search, scoring="log_loss", verbose=3, cv=1)

finalX = X[predictorsUsed].values.astype(np.float32)
finaly = y.values.astype(int)

print "pre Random search finished"

In [ ]:
print "starting Randomized Search"
start = time()
random_search.fit(finalX, finaly)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.grid_scores_)